In [1]:
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras import layers, Sequential
import numpy as np
import tensorflow as tf

2025-06-13 08:29:22.896646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749803363.067843      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749803363.120926      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset_dir = "/kaggle/input/stanford-dogs-dataset/images/Images"
batch_size = 32
img_height = 224
img_width = 224
validation_split = 0.3
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_test_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

num_val_test_batches = tf.data.experimental.cardinality(val_test_ds).numpy()
num_test_batches = int(num_val_test_batches * 0.5)
val_ds = val_test_ds.skip(num_test_batches)
test_ds = val_test_ds.take(num_test_batches)

Found 20580 files belonging to 120 classes.
Using 14406 files for training.


I0000 00:00:1749803395.746382      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 20580 files belonging to 120 classes.
Using 6174 files for validation.


In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

base_model = EfficientNetV2S(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
    classes=120
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(120, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[early_stopping]
)

82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/5


I0000 00:00:1749803444.404716     101 service.cc:148] XLA service 0x78fdec09ff70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749803444.405518     101 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1749803448.273035     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/451 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - accuracy: 0.0234 - loss: 4.8716   

I0000 00:00:1749803468.777896     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


451/451 ━━━━━━━━━━━━━━━━━━━━ 148s 204ms/step - accuracy: 0.6861 - loss: 1.7179 - val_accuracy: 0.9078 - val_loss: 0.3167
Epoch 2/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 34s 76ms/step - accuracy: 0.8832 - loss: 0.3821 - val_accuracy: 0.9142 - val_loss: 0.2960
Epoch 3/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 34s 76ms/step - accuracy: 0.9004 - loss: 0.3229 - val_accuracy: 0.9075 - val_loss: 0.3207
Epoch 4/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 34s 76ms/step - accuracy: 0.9087 - loss: 0.2818 - val_accuracy: 0.9072 - val_loss: 0.3210
Epoch 5/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 34s 76ms/step - accuracy: 0.9090 - loss: 0.2713 - val_accuracy: 0.9104 - val_loss: 0.3172


In [4]:
model.evaluate(test_ds)

96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.9050 - loss: 0.3200


[0.2959648668766022, 0.9091796875]

In [5]:
num_classes = len(train_ds.class_names)
class_names = train_ds.class_names

In [6]:
model.save('model-efficientnetv2s.keras')

In [7]:
maltese_url = "https://teketrek.net/wp-content/uploads/2024/05/%D9%83%D9%84%D8%A8-%D8%A7%D9%84%D9%85%D8%A7%D9%84%D8%B7%D9%8A.webp"
maltese_path = tf.keras.utils.get_file('Maltese', origin=maltese_url)

97714/97714 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
img = tf.keras.utils.load_img(
    maltese_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [9]:
predictions = model.predict(img_array)
score = predictions[0]

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
This image most likely belongs to n02085936-Maltese_dog with a 75.00 percent confidence.
